# Downloading COVID Data From FloridaDisaster.ORG

### To Do
- [ ] Turn all of the processes into functions
- [ ] Fix/find out why reports 3/19 - 3/24 are only 1kb and hold no data
- [ ] Clean the changing column names at the text export stage
- [ ] Figure out my headers are messed up
- [ ] 
- [ ] 
- [ ] 
- [X] Fix the problem where before doh reported 0 as 0 and not missing, I can't use the data
- [X] Extract the page range 12-25
- [X] Find the pages that contain the words "Coronavirus: All persons tested"
- [X] Extract just those two pages pdf 
- [X] Export individual files to txt for storage



In [8]:
from selenium import webdriver
import os
import wget
import PyPDF2 as pdf
from PyPDF2 import PdfFileReader, PdfFileWriter
import time
import textract
from pathlib import Path
import re
import pandas as pd
import datetime

In [ ]:
chrome_path = r"chromedriver.exe"

## Initializing driver element and finding 

In [ ]:
driver = webdriver.Chrome(chrome_path)

In [ ]:
driver.get('https://floridadisaster.org/covid19/covid-19-data-reports/')
title = driver.title
time.sleep(3)
result = driver.find_elements_by_xpath("//a[@href]")

In [4]:
output_path = Path('C:\\Users\\chacr\\OneDrive\\Documents\\Coding\\Python\\USF\\COVID-19 Project\\Data\\PDF')
pdf_output_path= output_path.resolve()

In [5]:
pdf_output_path

WindowsPath('C:/Users/chacr/OneDrive/Documents/Coding/Python/USF/COVID-19 Project/Data/PDF')

## Downloads the PDF Files from the FDEM Website

In [ ]:
link_counter = 0
len_result = len(result)
for links in result:
    if links.text.find('Data Report') > 0:
        link_url = links.get_attribute('href')
        filename = wget.filename_from_url(link_url)
        if not os.path.exists(f'{pdf_output_path}/{filename}'):
            wget.download(link_url, out = f'{pdf_output_path}')
        link_counter +=1
        print (f'{round((link_counter * 2)/len_result,3)*100}% Complete',end='\r')

### Extracting the min and max page range for the COVID-19 Daily County Data

In [ ]:
#turn into a function
page_counter = 0
today = datetime.date.today()
file_num = len(os.listdir(pdf_output_path))
progress = 0
for files in os.listdir(pdf_output_path):
    progress += 1
    output = f'Data/Unclean PDF/({today}) Unclean Extract from {files}'
    pdf_writer = PdfFileWriter()
    pdf_file = open(f'{pdf_output_path}/{files}','rb')
    pdfreader = PdfFileReader(pdf_file)
    num_pages = pdfreader.numPages
    if num_pages < 23:
        up_limit = num_pages
    else:
        up_limit = 23
    for pages in range(3, up_limit):
        page =  pdfreader.getPage(pages)  
        if page.extractText().find('Coronavirus: PUI testing by county') > 0 or page.extractText().find('Coronavirus: All persons with tests reported') > 0:
            pdf_writer.addPage(page)          
    with open(output, 'wb') as output_pdf:
        pdf_writer.write(output_pdf)
    print(f'{round(progress/file_num, ndigits=3)* 100}% Complete')

                                    

In [ ]:
# Converting the PDFs to html files

In [ ]:
import pdftotree


In [ ]:
input_path = Path('C:\\Users\\chacr\\OneDrive\\Documents\\Coding\\Python\\USF\\COVID-19 Project\\Data\\Unclean PDF')
output_path = Path('C:\\Users\\chacr\\OneDrive\\Documents\\Coding\\Python\\USF\\COVID-19 Project\\Data\\html')
ip = input_path.resolve()
op = output_path.resolve()

In [ ]:
print('Program started')
file_n = len(os.listdir(input_path))
file_ct = 0
error_list = []
date1 = r'^.*(\d+)-*(\d{2})-*(\d{4}).*\Z'
date2 = r'^.*(\d+)_(\d+)_(\d+).*\Z'
date3 = r'^.*(\d{4})-*(\d{2})-*(\d{2}).*\Z'
date4 = r'^.*(\d)\.(\d{2})\.(\d+).*\Z'
date5 = r'^.*(\d{2})_(\d{2})\Z'
last_file_name = ''
for files in os.listdir(ip):
    file_ct += 1
    if re.search(date3,files):
        new_file_name = re.sub(date3, r'html version of FL COVID Rep \2-\3-\1.html',files)    
    elif re.search(date1,files):
        new_file_name = re.sub(date1, r'html version of FL COVID Rep \1-\2-\3.html',files)
    elif re.search(date2,files):
        new_file_name = re.sub(date2, r'html version of FL COVID Rep \1-\2-\3.html',files)
    elif re.search(date4,files):
        new_file_name = re.sub(date4, r'html version of FL COVID Rep \1-\2-\3.html',files).replace('-20.html','-2020.html')
    elif re.search(date5,files):
        new_file_name = re.sub(date5, r'html version of FL COVID Rep \1-\2-2020.html',files)

    if last_file_name == new_file_name:
        new_file_name = new_file_name.replace('.html', '(PM).html')
    try:
        html = pdftotree.parse('C:\\Users\\chacr\\OneDrive\\Documents\\Coding\\Python\\USF\\COVID-19 Project\\Data\\Unclean PDF\\' +files)
        output_file_path = f'{op}/{new_file_name}.html'
        with open(output_file_path,'w') as f:
            f.write(html)
        last_file_name = new_file_name
#     except PDFSyntaxError as err:
#         error_list.append(files, err)
    except: 
        error_list.append(files)
    print(f'{round((file_ct/file_n)*100)}% Complete') 

## Importing the html files

In [23]:
import pandas as pd
import time
from bs4 import BeautifulSoup as soup

In [205]:
today = datetime.date.today()
html_path = Path('C:\\Users\\chacr\\OneDrive\\Documents\\Coding\\Python\\USF\\COVID-19 Project\\Data\\html').resolve()
txt_path = Path('C:\\Users\\chacr\\OneDrive\\Documents\\Coding\\Python\\USF\\COVID-19 Project\\Data\\TXT2').resolve()

In [289]:
final_df = pd.DataFrame(columns=['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED','FILE_NAME'])
for files in os.listdir(html_path):
    file_path = f'{html_path}/{files}'
    file =open(file_path,'r')
    page_soup = soup(file, 'lxml')
    table_soup = page_soup.findAll("table")
    row_counter = 0
    for tables in range(0, len(table_soup)):
        table_html = table_soup[tables]
        for row in table_html.findAll('tr'):
            row_length = len(row) -1
            counter = 1
            data_list = []
            for col in row.findAll('td'):

                if row_counter == 0:
                    data_list.append(col.text.replace(' ', '_').upper())
                else:
                    data_list.append(col.text)

                if counter == row_length and row_counter == 0:
                    print(row_length, row_counter)
                    if row_length == 5:
#                     if row_length == 5 and len(data_list[0]) == 0:
                        header = ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME']
                    elif row_length == 7 or row_length == 6: 
#                     elif row_length == 7 and len(data_list[0]) == 0:
                        header = ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED','FILE_NAME']
                    else:
                        print('ERROR: Header Length Not Correct')
                        break
                    
                    df = pd.DataFrame(columns=header)             

                if counter == (len(header)-1) and row_counter > 0 and data_list[0] != 'County' and len(data_list[0]) != 0:
                    print(files, header, data_list)
                    data_list.append(files.replace('.html',''))
                    data = pd.DataFrame([data_list], columns = header)
                    df = pd.concat([df,data])

                counter += 1
            row_counter += 1
    outfile = f'{txt_path}/{files.replace("html.html", "txt").replace("html version", "txt version")}'
    df.to_csv(outfile,sep='\t', index = False)
    final_df = final_df.append(df, ignore_index=True)

5 0
html version of FL COVID Rep 03-16-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Broward', '226', '153', '47', '11%', '']
html version of FL COVID Rep 03-16-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Dade', '174', '90', '25', '9%', '']
html version of FL COVID Rep 03-16-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '15', '25', '9', '18%', '']
html version of FL COVID Rep 03-16-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Collier', '37', '32', '7', '9%', '']
html version of FL COVID Rep 03-16-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Volusia', '11', '15', '7', '21%', '']
html version of FL COVI

5 0
html version of FL COVID Rep 03-18-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Dade', '148', '156', '76', '20%', '']
html version of FL COVID Rep 03-18-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '19', '28', '19', '29%', '']
html version of FL COVID Rep 03-18-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Collier', '53', '37', '13', '13%', '']
html version of FL COVID Rep 03-18-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Duval', '24', '40', '13', '17%', '']
html version of FL COVID Rep 03-18-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Hillsborough', '10', '42', '13', '20%', '']
html version of 

7 0
html version of FL COVID Rep 03-25-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Broward', '242', '', '0', '3,560', '412', '10%']
html version of FL COVID Rep 03-25-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '21', '', '0', '1,025', '141', '12%']
html version of FL COVID Rep 03-25-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Hillsborough', '10', '', '0', '1,773', '123', '6%']
html version of FL COVID Rep 03-25-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Orange', '22', '', '0', '1,112', '89', '7%']
html version of FL COVID Rep 03-25-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUS

html version of FL COVID Rep 03-25-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Total', '1,578', '', '3', '20,165', '1,977', '9%']
7 0
html version of FL COVID Rep 03-25-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Broward', '211', '', '0', '2,375', '355', '13%']
html version of FL COVID Rep 03-25-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '17', '', '0', '694', '118', '15%']
html version of FL COVID Rep 03-25-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Hillsborough', '8', '', '0', '1,474', '106', '7%']
html version of FL COVID Rep 03-25-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATI

html version of FL COVID Rep 03-26-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Duval', '287', '', '0', '1,104', '82', '7%']
html version of FL COVID Rep 03-26-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Collier', '71', '', '0', '627', '67', '10%']
html version of FL COVID Rep 03-26-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Lee', '30', '', '0', '695', '66', '9%']
html version of FL COVID Rep 03-26-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Pinellas', '54', '', '0', '1,111', '65', '6%']
html version of FL COVID Rep 03-26-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSIT

...

html version of FL COVID Rep 07-03-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Escambia', '2', '13', '33,137', '2,397', '7%', '']
html version of FL COVID Rep 07-03-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Brevard', '2', '5', '47,658', '2,366', '5%', '']
html version of FL COVID Rep 07-03-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Martin', '38', '22', '15,183', '2,208', '13%', '']
html version of FL COVID Rep 07-03-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['St. Lucie', '89', '11', '21,741', '1,983', '8%', '']
html version of FL COVID Rep 07-03-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE',

6 0
html version of FL COVID Rep 07-05-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Dade', '168', '442', '309,170', '47,011', '13%', '']
html version of FL COVID Rep 07-05-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Broward', '45', '113', '204,497', '21,239', '9%', '']
html version of FL COVID Rep 07-05-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '172', '250', '143,397', '16,836', '10%', '']
html version of FL COVID Rep 07-05-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Hillsborough', '119', '93', '127,191', '14,336', '10%', '']
html version of FL COVID Rep 07-05-2020.html.html ['COUNTY', 'AWAITING_TESTING', '

6 0
html version of FL COVID Rep 07-06-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Dade', '123', '211', '201,673', '20,872', '9%', '']
html version of FL COVID Rep 07-06-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Broward', '25', '73', '128,291', '8,589', '6%', '']
html version of FL COVID Rep 07-06-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '140', '239', '92,377', '8,209', '8%', '']
html version of FL COVID Rep 07-06-2020(PM).html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Hillsborough', '55', '80', '77,138', '3,295', '4%', '']
html version of FL COVID Rep 07-06-2020(PM).html.html ['COUNTY', 'AWAITING_T

6 0
html version of FL COVID Rep 07-06-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Dade', '166', '433', '302,528', '44,729', '13%', '']
html version of FL COVID Rep 07-06-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Broward', '44', '104', '197,948', '19,575', '9%', '']
html version of FL COVID Rep 07-06-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '173', '247', '140,088', '16,149', '10%', '']
html version of FL COVID Rep 07-06-2020.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Hillsborough', '117', '92', '124,603', '13,700', '10%', '']
html version of FL COVID Rep 07-06-2020.html.html ['COUNTY', 'AWAITING_TESTING', '

6 0
html version of FL COVID Rep 20-20-0606.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Dade', '74', '198', '180,521', '19,299', '10%', '']
html version of FL COVID Rep 20-20-0606.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Broward', '21', '62', '113,619', '7,690', '6%', '']
html version of FL COVID Rep 20-20-0606.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Palm Beach', '131', '234', '81,209', '7,074', '8%', '']
html version of FL COVID Rep 20-20-0606.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED', 'FILE_NAME'] ['Hillsborough', '31', '79', '66,805', '2,643', '4%', '']
html version of FL COVID Rep 20-20-0606.html.html ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSI

In [290]:
mod_df = final_df.fillna(0)

In [285]:
# del final_df 
# del df

In [291]:
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8245 entries, 0 to 8244
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   COUNTY            8245 non-null   object
 1   AWAITING_TESTING  8245 non-null   object
 2   INCONCLUSIVE      8245 non-null   object
 3   NEGATIVE          8245 non-null   object
 4   POSITIVE          8245 non-null   object
 5   PERCENT_POSITIVE  8245 non-null   object
 6   TOTAL_TESTED      8245 non-null   object
 7   FILE_NAME         8245 non-null   object
dtypes: object(8)
memory usage: 515.4+ KB


In [295]:
mod_df.describe()

,COUNTY,AWAITING_TESTING,INCONCLUSIVE,NEGATIVE,POSITIVE,PERCENT_POSITIVE,TOTAL_TESTED,FILE_NAME
count,8245,8245,8245,8245,8245,8245,8245,8245
unique,70,387,287,4169,1891,334,1144,116
top,Lee,,,0,0,3%,,html version of FL COVID Rep 03-18-2020
freq,125,1878,3480,1369,231,778,5356,385


In [222]:
print('Page soup length:', len(page_soup))
print('Page soup type:', type(page_soup))

Page soup length: 1
Page soup type: <class 'bs4.BeautifulSoup'>


In [223]:
print(page_soup)

<html><body><div id="1"><section_header bottom="108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 108.94348999999988 " char="C o r o n a v i r u s :   A l l   p e r s o n s   w i t h   t e s t s   r e p o r t e d " left="81.623 97.48

In [224]:
print(table_soup)

[<table><tr><td bottom="" char="" left="" right="" top=""></td><td bottom="212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 " char="A w a i t i n g   " left="173.01 179.091273 186.64578699999998 192.72706 195.485233 198.98556499999998 201.74373799999998 208.01332 213.540743 " right="178.947272 186.446401 192.50552 195.263693 198.896949 201.52219799999997 207.747472 213.452127 216.309993 " top="201.7176300000001 201.7176300000001 201.7176300000001 201.7176300000001 201.7176300000001 201.7176300000001 201.7176300000001 201.7176300000001 201.7176300000001 ">Awaiting</td><td bottom="" char="" left="" right="" top=""></td><td bottom="" char="" left="" right="" top=""></td><td bottom="" char="" left="" right="" top=""></td><td bottom="212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 212.7946300000001 " ch

In [233]:
row_counter = 0
for tables in range(0, len(table_soup)):
    table_html = table_soup[tables]
    for row in table_html.findAll('tr'):
        row_length = len(row) -1
        counter = 0
        data_list = []
        for col in row.findAll('td'):

            if row_counter == 0:
                data_list.append(col.text.replace(' ', '_').upper())
            else:
                data_list.append(col.text)
                
            if counter == row_length and row_counter == 0:
                if row_length == 6 and len(data_list[0]) == 0:
                    header = ['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED']
                else:
                    header = data_list
                df = pd.DataFrame(columns=header)
                
                
            if counter == row_length and row_counter > 0 and data_list[0] != 'County':
                data = pd.DataFrame([data_list], columns = header)
                df = pd.concat([df,data])
                
            counter += 1
        row_counter += 1
outfile = f'{txt_path}/{files.replace("html.html", "txt").replace("html version", "txt version")}'
df.to_csv(outfile,sep='\t', index = False)


['COUNTY', 'AWAITING_TESTING', 'INCONCLUSIVE', 'NEGATIVE', 'POSITIVE', 'PERCENT_POSITIVE', 'TOTAL_TESTED']


In [208]:
files.replace('html.html', 'txt')

'html version of FL COVID Rep 03-16-2020.txt'

In [254]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 0 to 0
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   COUNTY                 68 non-null     object
 1   AWAITING_TESTING       68 non-null     object
 2   INCONCLUSIVE           68 non-null     object
 3   NEGATIVE               68 non-null     object
 4   POSITIVE               68 non-null     object
 5   PERCENT_POSITIVE       68 non-null     object
 6   TOTAL_TESTEDFILE_NAME  68 non-null     object
dtypes: object(7)
memory usage: 4.2+ KB


In [203]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   COUNTY            1 non-null      object
 1   NO_RESULT         1 non-null      object
 2   NEGATIVE          1 non-null      object
 3   POSITIVE          1 non-null      object
 4   PERCENT_POSITIVE  1 non-null      object
 5   TOTAL             1 non-null      object
dtypes: object(6)
memory usage: 176.0+ bytes


In [182]:
test = pd.DataFrame([['1',2,3,4,5,6]] ,columns =header)

In [193]:
df2.append(test)

AttributeError: 'NoneType' object has no attribute 'is_extension'

In [190]:
print(df3)

NameError: name 'df3' is not defined

In [98]:
table_html.findAll('tr')

[<tr><td bottom="189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 " char="C o u n t y " left="82.476 89.126896 95.01548 101.10292799999999 107.190376 110.803072 " right="88.872792 94.87185600000001 101.003496 107.090944 110.59316 115.387992 " top="178.50112000000013 178.50112000000013 178.50112000000013 178.50112000000013 178.50112000000013 178.50112000000013 ">County</td><td bottom="189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 189.54912000000013 " char="N o   r e s u l t " left="164.6 172.013208 177.901792 180.939992 184.740504 190.629088 195.954224 202.041672 204.693192 " right="171.858536 177.75816799999998 180.663792 184.58583199999998 190.496512 195.75536 201.94224 204.58271200000001 208.095976 " top="178.50112000000013 178.50112000000013 178.50112000000013 178.50112000000013 178.50112000000013 178.5011200000001

In [111]:
test = data_list.copy()

In [113]:
print(test)

['Total', '848', '816', '160', '9%', '1,824']
